In [ ]:
epochs = 10
n_test_batches = 200

# भाग 11 - सुरक्षित डीप learning वर्गीकरण (Secure Deep Learning Classification)



## आपला डेटा महत्वाचा आहे, आपले मॉडेल देखील

मशीन लर्निंगमागील डेटा ड्रायव्हर आहे. डेटा तयार आणि संकलित करणार्‍या संस्था त्यांचे स्वत: चे मशीन शिक्षण मॉडेल तयार आणि प्रशिक्षित करण्यास सक्षम आहेत. हे त्यांना बाहेरील संस्थांना सर्व्हिस (MLaaS) म्हणून अशा मॉडेल्सचा वापर करण्याची ऑफर करण्यास परवानगी देते. हे इतर संस्था म्हणून उपयुक्त आहे जे कदाचित हे मॉडेल स्वतः तयार करु शकणार नाहीत परंतु तरीही त्यांच्या स्वत: च्या डेटावर अंदाज बांधण्यासाठी हे मॉडेल वापरू इच्छित आहेत.

तथापि, क्लाऊडमध्ये होस्ट केलेले मॉडेल अद्याप एक गोपनीयता / आयपी मुद्दा सादर करते. बाह्य संस्थांना ते वापरण्यासाठी - त्यांनी एकतर त्यांचा इनपुट डेटा (जसे की प्रतिमा वर्गीकृत करण्यासाठी प्रतिमा) अपलोड करणे आवश्यक आहे किंवा मॉडेल डाउनलोड करणे आवश्यक आहे. गोपनीयतेच्या दृष्टीकोनातून इनपुट डेटा अपलोड करणे त्रासदायक ठरू शकते, परंतु मॉडेल तयार / मालकीची संस्था आपला आयपी गमावण्याची चिंता करत असल्यास मॉडेल डाउनलोड करणे हा पर्याय असू शकत नाही.


## एन्क्रिप्टेड डेटा वर कम्प्यूटिंग

या संदर्भात, एक संभाव्य निराकरण म्हणजे मॉडेल आणि डेटा दोन्ही एन्क्रिप्ट करणे ज्यायोगे एखाद्या संस्थेने त्यांचे आयपी दुसर्‍याला न सांगता दुसर्‍या संस्थेच्या मालकीचे मॉडेल वापरण्याची परवानगी दिली. कित्येक एनक्रिप्शन योजना अस्तित्त्वात आहेत ज्या एन्क्रिप्टेड डेटापेक्षा संगणनास अनुमती देतात, त्यापैकी सिक्योर मल्टी-पार्टी कंप्यूटेशन (SMPC), होमोमॉर्फिक एन्क्रिप्शन (FHE/ SHE) आणि फंक्शनल एन्क्रिप्शन (FE) सर्वात प्रसिद्ध प्रकार आहेत. आम्ही येथे सिक्युर मल्टी-पार्टी कंप्यूटेशन ([ट्यूटोरियल 5 मध्ये तपशीलवार परिचय](https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%205%20-%20Intro%20to%20Encrypted%20Programs.ipynb)) वर लक्ष केंद्रित करूया ज्यात खाजगी अ‍ॅडिटीव्ह सामायिकरण आहे. हे SecureNN आणि SPDZ सारख्या क्रिप्टो प्रोटोकॉलवर अवलंबून आहे, ज्याचा तपशील [या उत्कृष्ट ब्लॉग पोस्टमध्ये देण्यात आला आहे](https://mortendahl.github.io/2017/09/19/pStreet-image-analysis-with-mpc/).

हे प्रोटोकॉल एन्क्रिप्टेड डेटापेक्षा उल्लेखनीय कामगिरी साध्य करतात आणि गेल्या काही महिन्यांपासून आम्ही हे प्रोटोकॉल वापरण्यास सुलभ करण्यासाठी कार्य करीत आहोत. विशेषतः, आपण स्वत: प्रोटोकॉलची पुन्हा अंमलबजावणी न करता आपल्याला हे प्रोटोकॉल वापरण्याची परवानगी देण्यासाठी साधने तयार करीत आहोत (किंवा हे कसे कार्य करते त्यामागील क्रिप्टोग्राफी देखील माहित असणे आवश्यक आहे). चला आत जाऊ.

## सेट अप

या ट्यूटोरियल मध्ये अचूक सेटिंग खालीलप्रमाणे आहेः आपण सर्व्हर आहात आणि आपल्याकडे काही डेटा आहे याचा विचार करा. प्रथम, आपण या खासगी प्रशिक्षण डेटासह मॉडेलची व्याख्या आणि प्रशिक्षण देता. मग, आपण एखाद्या क्लायंटशी संपर्क साधता ज्याचा स्वत: चा काही डेटा आहे जो आपल्या अंदाजात काही मॉडेलमध्ये प्रवेश करू इच्छितो.

आपण आपले मॉडेल एन्क्रिप्ट केले (एक neural नेटवर्क) क्लायंट त्यांचा डेटा कूटबद्ध करतो. आपण नंतर डेटाचे वर्गीकरण करण्यासाठी मॉडेल वापरण्यासाठी या दोन्ही कूटबद्ध मालमत्तांचा वापर करा. शेवटी, भविष्यवाणीचा परिणाम क्लायंटला पुन्हा एन्क्रिप्टेड मार्गाने पाठविला जातो जेणेकरून सर्व्हरला (म्हणजेच आपण) क्लायंटच्या डेटाविषयी काहीही शिकू शकत नाही (आपण इनपुट किंवा अंदाज दोघांनाही शिकत नाही).

तद्वतच आपण additively `client` चे इनपुट स्वतः आणि `server` दरम्यान सामायिक करू आणि त्याउलट मॉडेलसाठी. साधेपणासाठी, शेअर्स दोन इतर कामगार, `alice` आणि `bob` कडे असतील. जर आपल्याला हे समजले असेल की alice चा स्वामित्व क्लाइंट आहे आणि सर्व्हरद bob च्या मालकीची आहे तर ती पूर्णपणे समतुल्य आहे.

प्रामाणिक-परंतु-उत्सुक प्रतिरोधक मॉडेलमध्ये गणना सुरक्षित आहे जी [अनेक MPC फ्रेमवर्क](https://arxiv.org/pdf/1801.03239.pdf) मध्ये मानक आहे.

**आपल्याकडे आवश्यक असलेल्या सर्व गोष्टी आहेत, चला प्रारंभ करूया!**


लेखक:
- Théo Ryffel - Twitter: [@theoryffel](https://twitter.com/theoryffel) · GitHub: [@LaRiffle](https://github.com/LaRiffle)

अनुवादक/संपादक:
- Krunal Kshirsagar - Twitter: [@krunal_wrote](https://twitter.com/krunal_wrote) - Github: [@Noob-can-Compile](https://github.com/Noob-can-Compile)

**चला सुरू करुयात!**

### आयात आणि मॉडेल वैशिष्ट्ये

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

PySyft सुरू करण्यासाठी / आयात करण्यासाठी आपल्याला विशिष्ट आज्ञा अंमलात आणण्याची आवश्यकता आहे. आम्ही काही नावे असलेले (`client`,`bob`, और `alice`) कामगार तयार करतो. शेवटी, आपण `crypto_provider` परिभाषित करतो जी आपल्याला आवश्यक असलेल्या सर्व crypto primitives देते, ([अधिक तपशीलांसाठी आमचे SMPC वरील ट्यूटोरियल पहा](https://github.com/OpenMined/PySyft-blob/master/examples/tutorials/Parts/%2009%20%20Intro%20to%20Encrypted%20Programs.ipynb)).

In [ ]:
import syft as sy
hook = sy.TorchHook(torch) 
client = sy.VirtualWorker(hook, id="client")
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider") 

आपण शिकण्याच्या कार्याची सेटिंग परिभाषित करतो

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 50
        self.epochs = epochs
        self.lr = 0.001
        self.log_interval = 100

args = Arguments()

### डेटा लोड करणे आणि कामगारांना पाठवणे

डेटा लोड करणे आणि कामगारांना पाठवणे
आमच्या सेटिंगमध्ये आम्ही असे गृहित धरले की सर्व्हरला प्रथम मॉडेलचे प्रशिक्षण देण्यासाठी काही डेटामध्ये प्रवेश आहे. येथे MNIST प्रशिक्षण संच आहे.

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True)

दुसरे म्हणजे, क्लायंटकडे काही डेटा आहे आणि त्याबद्दल सर्व्हरचे मॉडेल वापरुन त्यावर अंदाज बांधणे आवडेल. हा ग्राहक `Alice` और `Bob` या दोन कामगारांना जोडून आपला डेटा सामायिक करतो.

> SMPC एक क्रिप्टो प्रोटोकॉल वापरतो ज्यास पूर्णांकांवर कार्य करणे आवश्यक आहे. आम्ही PyTorch Float टेन्सरला PySyft टेन्सर अ‍ॅबस्ट्रॅक्शनचा फायदा घेऊन `.fix_precision ()` वापरुन निश्चित परिशुद्धता टेन्सरमध्ये रुपांतरित करतो. उदाहरणार्थ, 0.123 हा अचूक २ सह दोन दशांश अंकांवर पूर्णांक आहे म्हणून संचयित संख्या पूर्णांक १२ आहे.

In [ ]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True)

private_test_loader = []
for data, target in test_loader:
    private_test_loader.append((
        data.fix_precision().share(alice, bob, crypto_provider=crypto_provider),
        target.fix_precision().share(alice, bob, crypto_provider=crypto_provider)
    ))

### फीड फॉरवर्ड न्यूरल नेटवर्क स्पेसिफिकेशन
सर्व्हरद्वारे वापरलेले नेटवर्क तपशील येथे आहे

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

### प्रशिक्षण शुरू करा

प्रशिक्षण स्थानिक पातळीवर केले जाते जेणेकरून हे शुद्ध स्थानिक PyTorch प्रशिक्षण आहे, येथे विशेष काही नाही!

In [ ]:
def train(args, model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        output = F.log_softmax(output, dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size,
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
    train(args, model, train_loader, optimizer, epoch)


In [ ]:
def test(args, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            output = F.log_softmax(output, dim=1)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
test(args, model, test_loader)

आमचे मॉडेल आता प्रशिक्षित आहे आणि सेवा म्हणून प्रदान करण्यास सज्ज आहे!

## सुरक्षित मूल्यांकन

आता सर्व्हर म्हणून आपण डेटा धारण करणार्‍या कामगारांना मॉडेल पाठवितो. कारण मॉडेल संवेदनशील माहिती आहे (आपण त्यास ऑप्टिमाइझ करण्यासाठी वेळ घालवला आहे!), आपण त्याचे वजन उघड करू इच्छित नाही जेणेकरून आपण आधी डेटासेटबरोबरच मॉडेल सामायिक केले.

In [ ]:
model.fix_precision().share(alice, bob, crypto_provider=crypto_provider)

हे चाचणी कार्य एन्क्रिप्टेड मूल्यांकन करते. स्कोरिंगसाठी वापरलेले मॉडेल वजन, डेटा इनपुट, भविष्यवाणी आणि लक्ष्य एन्क्रिप्ट केलेले आहे!

तथापि, syntax, मॉडेलच्या शुद्ध Pytorch चाचणीशी अगदी जुळली आहे, छान आहे ना ?!

अंदाजाची सरासरी चांगली होती हे सत्यापित करण्यासाठी आपण सर्व्हरच्या बाजूला असलेली एकमेव गोष्ट म्हणजे शेवटची धावसंख्या.

In [ ]:
def test(args, model, test_loader):
    model.eval()
    n_correct_priv = 0
    n_total = 0
    with torch.no_grad():
        for data, target in test_loader[:n_test_batches]:
            output = model(data)
            pred = output.argmax(dim=1) 
            n_correct_priv += pred.eq(target.view_as(pred)).sum()
            n_total += args.test_batch_size
# This 'test' function performs the encrypted evaluation. The model weights, the data inputs, the prediction and the target used for scoring are all encrypted!

# However as you can observe, the syntax is very similar to normal PyTorch testing! Nice!

# The only thing we decrypt from the server side is the final score at the end of our 200 items batches to verify predictions were on average good.      
            n_correct = n_correct_priv.copy().get().float_precision().long().item()
    
            print('Test set: Accuracy: {}/{} ({:.0f}%)'.format(
                n_correct, n_total,
                100. * n_correct / n_total))


In [ ]:
test(args, model, private_test_loader)

अभिनंदन! आपण येथे आहात, एंड-टू-एंड सुरक्षित भविष्यवाणी कसे करावे हे आपण शिकलात: सर्व्हरच्या मॉडेलचे वजन क्लायंटला कळाले नाही आणि सर्व्हरला डेटा इनपुट किंवा वर्गीकरण आउटपुटबद्दल माहिती नाही!

कार्यप्रदर्शनाबद्दल, माझ्या लॅपटॉपवर (2,7 गीगाहर्ट्ज इंटेल कोर i7, 16 जीबी रॅम), प्रतिमेचे वर्गीकरण करण्यासाठी ** 0.1 सेकंदांपेक्षा कमी ** लागतात जवळजवळ **33ms**. तथापि, हे अतिशय वेगवान संप्रेषण वापरत आहे (सर्व कामगार माझ्या स्थानिक मशीनवर आहेत). कार्यप्रदर्शन वेगवेगळे कामगार एकमेकांशी किती वेगवान बोलू शकतात यावर अवलंबून असते.

## निष्कर्ष

क्रिप्टोकर्न्सी तज्ञ नसताना, PyTorch आणि PySyft चा व्यावहारिक सुरक्षित मशीन शिक्षण घेण्यासाठी आणि वापरकर्त्यांचा डेटा सुरक्षित ठेवण्यासाठी किती सोपे आहे हे आपण पाहिले आहे!

या विषयावर लवकरच इतर ग्रंथालयांच्या संदर्भात बेंचमार्क PySyft योग्यरित्या कार्य करणे तसेच तंत्रिका नेटवर्कचे खाजगी एनक्रिप्टेड प्रशिक्षण समाविष्ट करणे आवश्यक आहे, जेव्हा एखादी संस्था बाह्य संवेदनशील डेटाचे स्वतःचे मॉडेल प्रशिक्षित करते तेव्हा आवश्यक आहे. समर्थन करत रहा! 

आपण आता स्वत: हून फेडरेटेड लर्निंग करण्यास सक्षम असावेत! आपण याचा आनंद घेत असल्यास आणि एआय आणि एआय सप्लाय चेन (डेटा) च्या विकेंद्रित मालकीच्या गोपनीयतेच्या संरक्षणाच्या दिशेने चळवळीत सामील होऊ इच्छित असाल तर आपण हे खालील प्रकारे करू शकता!

### Pysyft ला Github वर Star करा!

आमच्या समुदायाला मदत करण्याचा सर्वात सोपा मार्ग म्हणजे फक्त गिटहब(GitHub) रेपो(Repo) तारांकित(Star) करणे! हे आम्ही तयार करीत असलेल्या छान साधनांविषयी जागरूकता वाढविण्यास मदत करते.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### GitHub वरील आमचे प्रशिक्षण निवडा.

आम्ही फेडरेटेड आणि गोपनीयता-संरक्षित लर्निंगबद्दल अधिक चांगल्या प्रकारे समजवण्यासाठी खरोखर चांगले ट्यूटोरियल बनवले आहेत.

- [PySyft ट्यूटोरियल्सला चेक करा](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)

### आमच्या Slack मध्ये सामील व्हा!


नवीनतम प्रगतीवर अद्ययावत राहण्याचा उत्तम मार्ग म्हणजे आमच्या समुदायामध्ये सामील होणे! आपण [http://slack.openmined.org](http://slack.openmined.org) येथे फॉर्म भरुन तसे करू शकता.

### एका कोड प्रोजेक्टमध्ये सामील व्हा!

आमच्या समुदायामध्ये योगदानाचा उत्तम मार्ग म्हणजे कोड योगदानकर्ता बनणे! कोणत्याही वेळी आपण (PySyft GitHub Issues Page) वर  जाऊ शकता आणि "Project" साठी फिल्टर करू शकता. हे आपण कोणत्या प्रकल्पांमध्ये सामील होऊ शकता याबद्दल विहंगावलोकन देणारी सर्व उच्च स्तरीय तिकिटे दर्शवेल! आपण एखाद्या प्रकल्पात सामील होऊ इच्छित नसल्यास, परंतु आपण थोडं कोडिंग करू इच्छित असाल तर आपण "good first issues" म्हणून चिन्हांकित गिटहब(GitHub) अंक शोधून आणखी "one off" मिनी-प्रकल्प(mini project) शोधू शकता.

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### दान करा

आपल्याकडे आमच्या कोडेबेसमध्ये योगदान देण्यास वेळ नसल्यास, परंतु तरीही आपल्याला समर्थन द्यावयाचे असल्यास आपण आमच्या मुक्त संग्रहात बॅकर देखील होऊ शकता. सर्व देणगी आमच्या वेब होस्टिंग आणि हॅकॅथॉन आणि मेटअप्स सारख्या इतर सामुदायिक खर्चाकडे जातात!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)